In [18]:
import torch
from utils import get_son2parent
from utils import Node


hierarchy_csv = '/local/tlong/poincare_emb_2023/cifar100_hierarchy.csv'
emb_path = '/local/tlong/poincare_emb_2023/embs/cifar100_hierarchy_256d.pth'

son2parent = get_son2parent(hierarchy_csv)
emb_data = torch.load(emb_path)
embeddings = emb_data['embeddings']
objects = emb_data['objects']

root = Node('Root', son2parent)

# Recursively get the names of the nodes, keep only leaf nodes
def get_names(node):
    if node.children:
        return [get_names(item) for item in node.children]
    else:
        return node.name

names = get_names(root)
names = [item for sublist in names for item in sublist]


In [27]:
emb = torch.zeros(len(names), embeddings.shape[1])
for i, name in enumerate(names):
    emb[i] = embeddings[objects.index(name)]

In [46]:

import plotapi
# %matplotlib inline
# plotapi.api_key("1f5b3e32-b241-4425-bb4b-62aaa87ba87f")

from plotapi import Chord
  
# Chord.api_key("1f5b3e32-b241-4425-bb4b-62aaa87ba87f")
similarity = torch.mm(emb, emb.t()) / emb.norm(dim=1) / emb.norm(dim=1, keepdim=True)
similarity[similarity < 0.3] = 0
similarity = similarity - torch.diag(similarity.diag())

similarity = similarity.numpy().tolist()
# Visualization code here
Chord(similarity, names).to_html()

